# FFT Analysis with NumPy

This notebook covers frequency domain analysis using NumPy's FFT (Fast Fourier Transform) module.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available. Install with: pip install ipywidgets")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 4)

## 1. Introduction to the Fourier Transform

The Fourier Transform decomposes a signal into its frequency components. The FFT is an efficient algorithm for computing the Discrete Fourier Transform (DFT).

Key functions in `numpy.fft`:
- `fft()` - Compute the 1D FFT
- `ifft()` - Compute the inverse FFT
- `fftfreq()` - Get the frequency bins
- `rfft()` - FFT for real-valued input (returns only positive frequencies)
- `irfft()` - Inverse of rfft

In [2]:
def generate_sine_wave(frequency, amplitude=1.0, phase=0.0, duration=1.0, sample_rate=1000):
    """Generate a sine wave with specified parameters."""
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    y = amplitude * np.sin(2 * np.pi * frequency * t + phase)
    return t, y

## 2. Interactive FFT of a Sine Wave

Explore how changing the signal frequency affects the FFT output.

In [3]:
def plot_fft_sine(frequency=10, amplitude=1.0):
    """Plot a sine wave and its FFT."""
    sample_rate = 1000
    duration = 1.0
    
    t, signal = generate_sine_wave(frequency=frequency, amplitude=amplitude,
                                    duration=duration, sample_rate=sample_rate)
    n_samples = len(signal)
    
    # Compute FFT
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    magnitude = np.abs(fft_result) / n_samples * 2
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(t, signal, 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'{frequency} Hz Sine Wave (A={amplitude})')
    axes[0].set_ylim(-2.5, 2.5)
    axes[0].grid(True)
    
    axes[1].stem(frequencies, magnitude)
    axes[1].axvline(x=frequency, color='r', linestyle='--', alpha=0.5, label=f'Expected: {frequency} Hz')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title('FFT Magnitude Spectrum')
    axes[1].set_xlim(0, 100)
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Find peak
    peak_idx = np.argmax(magnitude)
    print(f"Detected peak at: {frequencies[peak_idx]:.1f} Hz with amplitude {magnitude[peak_idx]:.2f}")

if WIDGETS_AVAILABLE:
    interact(
        plot_fft_sine,
        frequency=IntSlider(min=1, max=50, value=10, description='Frequency (Hz)'),
        amplitude=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Amplitude')
    )
else:
    plot_fft_sine()

interactive(children=(IntSlider(value=10, description='Frequency (Hz)', max=50, min=1), FloatSlider(value=1.0,…

## 3. Magnitude and Phase Spectrum

The FFT returns complex numbers. The magnitude tells us the strength of each frequency component, while the phase tells us the time offset.

In [4]:
def plot_magnitude_phase(frequency=10, phase=0.0):
    """Explore magnitude and phase spectra."""
    sample_rate = 1000
    duration = 1.0
    
    t, signal = generate_sine_wave(frequency=frequency, phase=phase,
                                    duration=duration, sample_rate=sample_rate)
    n_samples = len(signal)
    
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    magnitude = np.abs(fft_result) / n_samples * 2
    phase_spectrum = np.angle(fft_result)
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
    axes[0].plot(t, signal, 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Signal: {frequency} Hz, φ={phase:.2f} rad')
    axes[0].grid(True)
    
    axes[1].stem(frequencies[:50], magnitude[:50])
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title('Magnitude Spectrum')
    axes[1].grid(True)
    
    axes[2].stem(frequencies[:50], phase_spectrum[:50])
    axes[2].set_xlabel('Frequency (Hz)')
    axes[2].set_ylabel('Phase (radians)')
    axes[2].set_title('Phase Spectrum')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Show phase at signal frequency
    freq_idx = np.argmin(np.abs(frequencies - frequency))
    measured_phase = phase_spectrum[freq_idx]
    # Adjust for sine vs cosine reference
    expected_phase = phase - np.pi/2
    print(f"Phase at {frequency} Hz: {measured_phase:.2f} rad ({np.degrees(measured_phase):.0f}°)")

if WIDGETS_AVAILABLE:
    interact(
        plot_magnitude_phase,
        frequency=IntSlider(min=5, max=30, value=10, description='Frequency (Hz)'),
        phase=FloatSlider(min=0, max=2*np.pi, step=0.1, value=0, description='Phase (rad)')
    )
else:
    plot_magnitude_phase()

interactive(children=(IntSlider(value=10, description='Frequency (Hz)', max=30, min=5), FloatSlider(value=0.0,…

## 4. Analyzing Composite Signals

The power of FFT: it can identify all frequency components in a complex signal.

In [5]:
def plot_composite_fft(f1=5, a1=1.0, f2=20, a2=0.5, f3=50, a3=0.3):
    """Analyze a composite signal with multiple frequencies."""
    sample_rate = 1000
    duration = 1.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    signal = (a1 * np.sin(2 * np.pi * f1 * t) + 
              a2 * np.sin(2 * np.pi * f2 * t) + 
              a3 * np.sin(2 * np.pi * f3 * t))
    
    n_samples = len(signal)
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    magnitude = np.abs(fft_result) / n_samples * 2
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(t, signal, 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title('Composite Signal (Time Domain)')
    axes[0].grid(True)
    
    axes[1].stem(frequencies, magnitude)
    axes[1].axvline(x=f1, color='r', linestyle='--', alpha=0.5, label=f'{f1} Hz (A={a1})')
    axes[1].axvline(x=f2, color='g', linestyle='--', alpha=0.5, label=f'{f2} Hz (A={a2})')
    axes[1].axvline(x=f3, color='b', linestyle='--', alpha=0.5, label=f'{f3} Hz (A={a3})')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title('FFT: Detected Frequencies')
    axes[1].set_xlim(0, 80)
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        plot_composite_fft,
        f1=IntSlider(min=1, max=20, value=5, description='Freq 1 (Hz)'),
        a1=FloatSlider(min=0, max=1.5, step=0.1, value=1.0, description='Amp 1'),
        f2=IntSlider(min=10, max=40, value=20, description='Freq 2 (Hz)'),
        a2=FloatSlider(min=0, max=1.5, step=0.1, value=0.5, description='Amp 2'),
        f3=IntSlider(min=30, max=70, value=50, description='Freq 3 (Hz)'),
        a3=FloatSlider(min=0, max=1.5, step=0.1, value=0.3, description='Amp 3')
    )
else:
    plot_composite_fft()

interactive(children=(IntSlider(value=5, description='Freq 1 (Hz)', max=20, min=1), FloatSlider(value=1.0, des…

## 5. Frequency Resolution and Signal Duration

The frequency resolution of the FFT depends on the signal duration:
- Frequency resolution = 1 / duration = sample_rate / n_samples

In [6]:
def plot_frequency_resolution(frequency=7.5, duration=1.0):
    """Explore how signal duration affects frequency resolution."""
    sample_rate = 100
    
    t, signal = generate_sine_wave(frequency=frequency, duration=duration, 
                                    sample_rate=sample_rate)
    n_samples = len(signal)
    
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    magnitude = np.abs(fft_result) / n_samples * 2
    
    freq_resolution = sample_rate / n_samples
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(t, signal, 'b-')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Signal: {frequency} Hz, Duration: {duration}s')
    axes[0].grid(True)
    
    axes[1].stem(frequencies, magnitude)
    axes[1].axvline(x=frequency, color='r', linestyle='--', alpha=0.5, 
                    label=f'True freq: {frequency} Hz')
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Magnitude')
    axes[1].set_title(f'FFT (Resolution: {freq_resolution:.2f} Hz)')
    axes[1].set_xlim(0, 20)
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Frequency resolution: {freq_resolution:.2f} Hz")
    print(f"Nearest FFT bin to {frequency} Hz: {frequencies[np.argmin(np.abs(frequencies - frequency))]:.2f} Hz")

if WIDGETS_AVAILABLE:
    interact(
        plot_frequency_resolution,
        frequency=FloatSlider(min=5, max=15, step=0.5, value=7.5, description='Frequency (Hz)'),
        duration=FloatSlider(min=0.5, max=4.0, step=0.5, value=1.0, description='Duration (s)')
    )
else:
    plot_frequency_resolution()

interactive(children=(FloatSlider(value=7.5, description='Frequency (Hz)', max=15.0, min=5.0, step=0.5), Float…

## 6. Inverse FFT: Reconstructing Signals

We can reconstruct the original signal from its FFT using the inverse FFT.

In [7]:
def plot_reconstruction(frequency=10, keep_components=50):
    """Demonstrate signal reconstruction with limited frequency components."""
    sample_rate = 1000
    duration = 1.0
    
    t, signal = generate_sine_wave(frequency=frequency, duration=duration, 
                                    sample_rate=sample_rate)
    n_samples = len(signal)
    
    # Add some noise
    np.random.seed(42)
    noisy_signal = signal + 0.3 * np.random.randn(n_samples)
    
    # FFT
    fft_result = np.fft.rfft(noisy_signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    # Keep only some components
    fft_truncated = fft_result.copy()
    fft_truncated[keep_components:] = 0
    
    # Reconstruct
    reconstructed = np.fft.irfft(fft_truncated)
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 6))
    
    axes[0].plot(t, noisy_signal, 'b-', alpha=0.5, label='Noisy signal')
    axes[0].plot(t, signal, 'g--', linewidth=2, label='Original')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title('Original and Noisy Signal')
    axes[0].legend()
    axes[0].grid(True)
    
    axes[1].plot(t, noisy_signal, 'b-', alpha=0.3, label='Noisy')
    axes[1].plot(t, reconstructed, 'r-', linewidth=2, 
                 label=f'Reconstructed ({keep_components} components)')
    axes[1].plot(t, signal, 'g--', alpha=0.5, label='Original')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude')
    axes[1].set_title('Reconstruction with Limited FFT Components')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    error = np.sqrt(np.mean((signal - reconstructed)**2))
    print(f"RMS error vs original: {error:.4f}")

if WIDGETS_AVAILABLE:
    interact(
        plot_reconstruction,
        frequency=IntSlider(min=5, max=30, value=10, description='Frequency (Hz)'),
        keep_components=IntSlider(min=1, max=200, value=50, description='# Components')
    )
else:
    plot_reconstruction()

interactive(children=(IntSlider(value=10, description='Frequency (Hz)', max=30, min=5), IntSlider(value=50, de…

## 7. Power Spectral Density

The Power Spectral Density (PSD) shows the distribution of power across frequencies.

In [8]:
def plot_psd(f1=10, f2=25, f3=50, use_db_scale=True):
    """Explore power spectral density."""
    sample_rate = 1000
    duration = 2.0
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    signal = (1.0 * np.sin(2 * np.pi * f1 * t) + 
              0.5 * np.sin(2 * np.pi * f2 * t) + 
              0.3 * np.sin(2 * np.pi * f3 * t))
    
    n_samples = len(signal)
    fft_result = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(n_samples, d=1/sample_rate)
    
    psd = (np.abs(fft_result) ** 2) / n_samples
    
    fig, ax = plt.subplots(figsize=(10, 4))
    
    if use_db_scale:
        psd_plot = 10 * np.log10(psd + 1e-10)
        ylabel = 'Power (dB)'
    else:
        psd_plot = psd
        ylabel = 'Power'
    
    ax.plot(frequencies, psd_plot, 'b-')
    ax.axvline(x=f1, color='r', linestyle='--', alpha=0.5, label=f'{f1} Hz')
    ax.axvline(x=f2, color='g', linestyle='--', alpha=0.5, label=f'{f2} Hz')
    ax.axvline(x=f3, color='orange', linestyle='--', alpha=0.5, label=f'{f3} Hz')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel(ylabel)
    ax.set_title('Power Spectral Density')
    ax.set_xlim(0, 80)
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

if WIDGETS_AVAILABLE:
    interact(
        plot_psd,
        f1=IntSlider(min=5, max=20, value=10, description='Freq 1 (Hz)'),
        f2=IntSlider(min=15, max=40, value=25, description='Freq 2 (Hz)'),
        f3=IntSlider(min=35, max=70, value=50, description='Freq 3 (Hz)'),
        use_db_scale=Checkbox(value=True, description='dB Scale')
    )
else:
    plot_psd()

interactive(children=(IntSlider(value=10, description='Freq 1 (Hz)', max=20, min=5), IntSlider(value=25, descr…

## Summary

In this notebook, we covered:
- Computing the FFT with `np.fft.fft()` and `np.fft.rfft()`
- Interpreting magnitude and phase spectra
- Using `fftfreq()` to get frequency bins
- Analyzing signals with multiple frequency components
- Understanding frequency resolution
- Reconstructing signals with inverse FFT
- Power spectral density

Next, in **03_practical_applications.ipynb**, we'll explore real-world applications including noise filtering and windowing.